In [4]:
from neo4j import GraphDatabase
import pickle
import pandas as pd

### When using MERGE or MATCH with LOAD CSV we need to make sure we have an index or a unique constraint on the property that we are merging on. This will ensure that the query executes in a performant way.

- CREATE INDEX ON :Teller(id)
- CREATE INDEX ON :Account(id)

In [6]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "lv23623600"))

## Add Hall (营业机构)

In [7]:
teller = pd.read_csv("teller.csv", index_col = 0) 
institutes = list(teller["YNGYJIGO"].unique())

In [10]:
def add_hall(tx):
    """add haller
    """
    for ins in institutes:
        tx.run("CREATE (ins: Hall{{ id:{} }})".format(ins))

In [11]:
with driver.session() as session:
    session.write_transaction(add_hall)

## Add Teller (柜员)

In [12]:
teller.head()

,GUIYDAIH,YNGYJIGO,level
0,001041,601101,1
1,004053,601104,1
2,007020,601129,1
3,015005,601104,0
4,011024,601114,1


In [13]:
def add_teller(tx):
    """add teller
    """
    for line in teller.values.tolist():
        if line[2] == 0:
            tx.run("CREATE (te: Teller{{ id: \"{}\", hall: {}, level: {} }})".format(line[0],line[1],line[2]))        

In [14]:
with driver.session() as session:
    session.write_transaction(add_teller)

## add cheif

In [15]:
def add_cheif(tx):
    """add teller
    """
    for line in teller.values.tolist():
        if line[2] == 1:
            tx.run("CREATE (te: Cheif{{ id: \"{}\", hall: {}, level: {} }})".format(line[0],line[1],line[2]))        

In [16]:
with driver.session() as session:
    session.write_transaction(add_cheif)

## add customer (客户号)

In [17]:
def add_customer(tx):
    """add customer
    """
    tx.run("LOAD CSV WITH HEADERS  FROM \"file:///customer_tb.csv\" AS line MERGE (customer:Customer{id: toInt(line.CUST_CODE),C_GENDER:line.C_GENDER,C_AGE:line.C_AGE,C_FIRST_WORK_DATE:line.C_FIRST_WORK_DATE })")

In [18]:
with driver.session() as session:
    session.write_transaction(add_customer)

## add account (账户号)

In [19]:
cust_account = pd.read_csv("cust_acct.csv", index_col = 0)
cust_account.head()

,CUST_ID,ACCT_ID
0,39697816,2265871694
1,39697816,4567905843
2,39697816,4211888405
3,39697816,3241956117
4,39697816,5170732060


In [20]:
all_account = list(cust_account["ACCT_ID"].unique())

In [21]:
def add_account(tx):
    """add account
    """
    for acct_id in all_account:
        tx.run("CREATE (ac: Account{{ id: {} }})".format(acct_id))

In [22]:
with driver.session() as session:
    session.write_transaction(add_account)

## add President

In [48]:
def add_president(tx):
    """add President
    """
    tx.run("CREATE (p: President{ id: '000000' })")

with driver.session() as session:
    session.write_transaction(add_president)

## add index

In [23]:
def add_index(tx):
    tx.run("CREATE INDEX ON :Teller(id)")
    tx.run("CREATE INDEX ON :Account(id)")

In [24]:
with driver.session() as session:
    session.write_transaction(add_index)

## add belongTo relation (Customer has many account)

In [25]:
def add_belongTo_rela(tx):
    """add belongTo relation, some customer has many 
    """
    tx.run("LOAD CSV WITH HEADERS  FROM \"file:///cust_acct.csv\" AS line match (from:Account{id: toInt(line.ACCT_ID) }),(to:Customer{id: toInt(line.CUST_ID)}) merge (from)-[r:belongTo]->(to)")

In [26]:
with driver.session() as session:
    session.write_transaction(add_belongTo_rela)

## add transaction relation (account -> account)

In [27]:
transaction = pd.read_csv("transaction.csv", index_col = 0)
transaction.head()

,TR_TM,TR_NO,OPR_ID,CUST_ID,ACCT_ID,TR_AM,OPP_ACCT_ID
0,15-AUG-18,IN00110117000044,IN0011,49744050,9422937241,2215,4161831387
1,15-AUG-18,IN00290117000025,IN0029,88528785,2553396999,169684,8852694812
2,15-AUG-18,IN00110117000010,IN0011,63334672,3842340517,856189,8518307638
3,15-AUG-18,IN00170117000076,IN0017,19815276,5174142869,204857,7259062305
4,15-AUG-18,IN00070117000031,IN0007,44407636,4809166321,548499,9041482456


In [28]:
def add_transaction_rela(tx):
    """add transaction relation (a account transfer money to another account)
    """
    tx.run("LOAD CSV WITH HEADERS  FROM \"file:///transaction.csv\" AS line match (from:Account{id: toInt(line.ACCT_ID) }),(to:Account{id: toInt(line.OPP_ACCT_ID) }) merge (from)-[r:transaction{ TR_TM: line.TR_TM, TR_NO: line.TR_NO, TR_AM: toInt(line.TR_AM), OPR_id: line.OPR_ID }]->(to)")

In [29]:
with driver.session() as session:
    session.write_transaction(add_transaction_rela)

## add operate relation (teller can server many account )

In [30]:
def add_serve_rela(tx):
    """add serve relation (teller can serve many account )
    """
    tx.run("LOAD CSV WITH HEADERS  FROM \"file:///transaction.csv\" AS line match (from:Teller{id: line.OPR_ID}),(to:Account{id: toInt(line.ACCT_ID)}) merge (from)-[r:operate]->(to)")
    

In [31]:
with driver.session() as session:
    session.write_transaction(add_serve_rela)

## add manage relation (cheif manage institute)

In [34]:
teller.head()

,GUIYDAIH,YNGYJIGO,level
0,001041,601101,1
1,004053,601104,1
2,007020,601129,1
3,015005,601104,0
4,011024,601114,1


In [45]:
def add_manage_rela(tx):
    for line in teller.values.tolist():
        if line[-1] == 1:
            tx.run("MATCH (c:Cheif {{id: \"{}\" }}), (h:Hall {{ id:{} }}) merge (c)-[:manage]->(h)".format(line[0],line[1]))
            #print("MATCH (c:Cheif {{id: \"{}\" }}), (h:Hall {{ id: {} }}) merge (c)-[r:manage]->(h)".format(line[0],line[1]))

In [46]:
with driver.session() as session:
    session.write_transaction(add_manage_rela)

## add rule president can rule all the Hall

In [52]:
def add_rule_rela(tx):
    tx.run("MATCH (h:Hall),(p:President) merge (p)-[:rule]->(h)")

In [53]:
with driver.session() as session:
    session.write_transaction(add_rule_rela)

## add workIn relation

In [54]:
def add_workIn_rela(tx):
    tx.run("MATCH (h:Hall),(t:Teller) WHERE h.id = t.hall MERGE (t)-[:workIn]->(h)")

In [55]:
with driver.session() as session:
    session.write_transaction(add_workIn_rela)